# Tutorial: Build a regression model with Open Datasets

In this tutorial, you leverage the convenience of Azure Open Datasets to create a regression model to predict NYC taxi fare prices. Easily download publicly available taxi, holiday and weather data to create a dataset that can train a regression model using sklearn.

In [4]:
from azureml.opendatasets import NycTlcGreen
import pandas as pd
import numpy as np
from datetime import datetime
from dateutil.relativedelta import relativedelta

pd.options.mode.chained_assignment = None

Failure while loading azureml_run_type_providers. Failed to load entrypoint hyperdrive = azureml.train.hyperdrive:HyperDriveRun._from_run_dto with exception (azureml-telemetry 1.39.0 (c:\users\fkhoshouei\anaconda3\lib\site-packages), Requirement.parse('azureml-telemetry~=1.28.0')).
Failure while loading azureml_run_type_providers. Failed to load entrypoint automl = azureml.train.automl.run:AutoMLRun._from_run_dto with exception (azureml-dataset-runtime 1.39.0 (c:\users\fkhoshouei\anaconda3\lib\site-packages), Requirement.parse('azureml-dataset-runtime~=1.28.0')).
Failure while loading azureml_run_type_providers. Failed to load entrypoint azureml.PipelineRun = azureml.pipeline.core.run:PipelineRun._from_dto with exception (azureml-core 1.39.0.post1 (c:\users\fkhoshouei\anaconda3\lib\site-packages), Requirement.parse('azureml-core~=1.28.0')).
Failure while loading azureml_run_type_providers. Failed to load entrypoint azureml.ReusedStepRun = azureml.pipeline.core.run:StepRun._from_reused_

## Download Data
Begin by downloading the NYC Taxi dataset from Azure Open Datasets. In non-Spark environments, Open Datasets only allows one month of data at a time with certain classes to avoid MemoryError with large datasets. To download 1 year of taxi data, we will fetch 2000 random samples from each month.

Note: Open Datasets has mirroring classes for working in Spark where data size and memory are not a concern.

In [5]:
start = datetime.strptime("1/1/2016","%m/%d/%Y")
end = datetime.strptime("1/31/2016","%m/%d/%Y")

green_taxi_df = pd.concat([NycTlcGreen(start + relativedelta(months=x), end + relativedelta(months=x)) \
        .to_pandas_dataframe().sample(2000) for x in range(12)])
green_taxi_df

[Info] read from C:\Users\FKHOSH~1\AppData\Local\Temp\tmpqk6fyapa\https%3A\%2Fazureopendatastorage.azurefd.net\nyctlc\green\puYear=2016\puMonth=1\part-00119-tid-4753095944193949832-fee7e113-666d-4114-9fcb-bcd3046479f3-2689-1.c000.snappy.parquet
[Info] read from C:\Users\FKHOSH~1\AppData\Local\Temp\tmppe5b440l\https%3A\%2Fazureopendatastorage.azurefd.net\nyctlc\green\puYear=2016\puMonth=2\part-00060-tid-4753095944193949832-fee7e113-666d-4114-9fcb-bcd3046479f3-2630-2.c000.snappy.parquet
[Info] read from C:\Users\FKHOSH~1\AppData\Local\Temp\tmphrkt_r3x\https%3A\%2Fazureopendatastorage.azurefd.net\nyctlc\green\puYear=2016\puMonth=3\part-00196-tid-4753095944193949832-fee7e113-666d-4114-9fcb-bcd3046479f3-2766-1.c000.snappy.parquet
[Info] read from C:\Users\FKHOSH~1\AppData\Local\Temp\tmp_vlfl53x\https%3A\%2Fazureopendatastorage.azurefd.net\nyctlc\green\puYear=2016\puMonth=4\part-00121-tid-4753095944193949832-fee7e113-666d-4114-9fcb-bcd3046479f3-2691-1.c000.snappy.parquet
[Info] read from C:\

,vendorID,lpepPickupDatetime,lpepDropoffDatetime,passengerCount,tripDistance,puLocationId,doLocationId,pickupLongitude,pickupLatitude,dropoffLongitude,...,paymentType,fareAmount,extra,mtaTax,improvementSurcharge,tipAmount,tollsAmount,ehailFee,totalAmount,tripType
425544,2,2016-01-07 20:16:56,2016-01-07 20:31:32,1,2.99,None,None,-73.964279,40.682938,-73.957130,...,2,12.0,0.5,0.5,0.3,0.00,0.00,NaN,13.30,1.0
978803,2,2016-01-21 00:41:54,2016-01-21 00:48:14,1,1.41,None,None,-73.890945,40.746464,-73.887543,...,2,7.0,0.5,0.5,0.3,0.00,0.00,NaN,8.30,1.0
452829,2,2016-01-08 13:53:04,2016-01-08 13:58:47,1,1.29,None,None,-73.867073,40.831944,-73.882149,...,2,6.5,0.0,0.5,0.3,0.00,0.00,NaN,7.30,1.0
49731,2,2016-01-04 17:13:18,2016-01-04 17:14:02,1,0.08,None,None,-73.935860,40.806469,-73.936714,...,3,-2.5,-1.0,-0.5,-0.3,0.00,0.00,NaN,-4.30,1.0
231441,2,2016-01-29 18:06:02,2016-01-29 18:14:19,1,1.79,None,None,-73.831833,40.715412,-73.816788,...,2,8.0,1.0,0.5,0.3,0.00,0.00,NaN,9.80,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1106380,2,2016-12-07 22:47:08,2016-12-07 23:18:39,1,6.05,7,144,NaN,NaN,NaN,...,2,25.0,0.5,0.5,0.3,0.00,0.00,NaN,26.30,1.0
940796,1,2016-12-20 07:48:08,2016-12-20 07:59:39,1,2.50,145,233,NaN,NaN,NaN,...,2,10.0,0.0,0.5,0.3,0.00,5.54,NaN,16.34,1.0
1110880,1,2016-12-08 00:01:02,2016-12-08 00:07:52,1,1.30,181,61,NaN,NaN,NaN,...,1,7.0,0.5,0.5,0.3,2.05,0.00,NaN,10.35,1.0
1027812,1,2016-12-26 00:38:06,2016-12-26 00:44:29,1,1.80,7,226,NaN,NaN,NaN,...,2,7.5,0.5,0.5,0.3,0.00,0.00,NaN,8.80,1.0


Now that the initial data is loaded, define a function to create various time-based features from the pickup datetime field. This will create new fields for the month number, day of month, day of week, and hour of day. From those, we calculate the sin and cosine transformations to capture the cyclical nature of the variable which will allow the model to factor in time-based seasonality. This function also adds a static feature for the country code to join the holiday data. Use the apply() function on the dataframe to interatively apply this function to each row in the dataframe.

In [6]:
def build_time_features(vector):
    pickup_datetime = vector[0]
    month_num = pickup_datetime.month
    day_of_month = pickup_datetime.day
    day_of_week = pickup_datetime.weekday()
    hour_of_day = pickup_datetime.hour
    country_code = "US"
    hr_sin = np.sin(hour_of_day*(2.*np.pi/24))
    hr_cos = np.cos(hour_of_day*(2.*np.pi/24))
    dy_sin = np.sin(day_of_week*(2.*np.pi/7))
    dy_cos = np.cos(day_of_week*(2.*np.pi/7))
    
    return pd.Series((month_num, day_of_month, day_of_week, hour_of_day, country_code, hr_sin, hr_cos, dy_sin, dy_cos))

green_taxi_df[["month_num", "day_of_month","day_of_week", "hour_of_day", "country_code", "hr_sin", "hr_cos", "dy_sin", "dy_cos"]] = green_taxi_df[["lpepPickupDatetime"]].apply(build_time_features, axis=1)
green_taxi_df

,vendorID,lpepPickupDatetime,lpepDropoffDatetime,passengerCount,tripDistance,puLocationId,doLocationId,pickupLongitude,pickupLatitude,dropoffLongitude,...,tripType,month_num,day_of_month,day_of_week,hour_of_day,country_code,hr_sin,hr_cos,dy_sin,dy_cos
425544,2,2016-01-07 20:16:56,2016-01-07 20:31:32,1,2.99,None,None,-73.964279,40.682938,-73.957130,...,1.0,1,7,3,20,US,-0.866025,5.000000e-01,0.433884,-0.900969
978803,2,2016-01-21 00:41:54,2016-01-21 00:48:14,1,1.41,None,None,-73.890945,40.746464,-73.887543,...,1.0,1,21,3,0,US,0.000000,1.000000e+00,0.433884,-0.900969
452829,2,2016-01-08 13:53:04,2016-01-08 13:58:47,1,1.29,None,None,-73.867073,40.831944,-73.882149,...,1.0,1,8,4,13,US,-0.258819,-9.659258e-01,-0.433884,-0.900969
49731,2,2016-01-04 17:13:18,2016-01-04 17:14:02,1,0.08,None,None,-73.935860,40.806469,-73.936714,...,1.0,1,4,0,17,US,-0.965926,-2.588190e-01,0.000000,1.000000
231441,2,2016-01-29 18:06:02,2016-01-29 18:14:19,1,1.79,None,None,-73.831833,40.715412,-73.816788,...,1.0,1,29,4,18,US,-1.000000,-1.836970e-16,-0.433884,-0.900969
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1106380,2,2016-12-07 22:47:08,2016-12-07 23:18:39,1,6.05,7,144,NaN,NaN,NaN,...,1.0,12,7,2,22,US,-0.500000,8.660254e-01,0.974928,-0.222521
940796,1,2016-12-20 07:48:08,2016-12-20 07:59:39,1,2.50,145,233,NaN,NaN,NaN,...,1.0,12,20,1,7,US,0.965926,-2.588190e-01,0.781831,0.623490
1110880,1,2016-12-08 00:01:02,2016-12-08 00:07:52,1,1.30,181,61,NaN,NaN,NaN,...,1.0,12,8,3,0,US,0.000000,1.000000e+00,0.433884,-0.900969
1027812,1,2016-12-26 00:38:06,2016-12-26 00:44:29,1,1.80,7,226,NaN,NaN,NaN,...,1.0,12,26,0,0,US,0.000000,1.000000e+00,0.000000,1.000000


Remove some of the columns that you won't need for modeling or additional feature building. Rename the time field for pickup time, and additionally convert the time to midnight using `pandas.Series.dt.normalize`. This is done to all time features so that the datetime column can be later used as a key when joining datasets together at a daily level of granularity.

In [7]:
columns_to_remove = ["lpepDropoffDatetime", "puLocationId", "doLocationId", "extra", "mtaTax",
                     "improvementSurcharge", "tollsAmount", "ehailFee", "tripType", "rateCodeID", 
                     "storeAndFwdFlag", "paymentType", "fareAmount", "tipAmount"]

green_taxi_df.drop(columns_to_remove, axis=1, inplace=True)

green_taxi_df["datetime"] = green_taxi_df["lpepPickupDatetime"].dt.normalize()
green_taxi_df.head(5)

,vendorID,lpepPickupDatetime,passengerCount,tripDistance,pickupLongitude,pickupLatitude,dropoffLongitude,dropoffLatitude,totalAmount,month_num,day_of_month,day_of_week,hour_of_day,country_code,hr_sin,hr_cos,dy_sin,dy_cos,datetime
425544,2,2016-01-07 20:16:56,1,2.99,-73.964279,40.682938,-73.957130,40.712856,13.3,1,7,3,20,US,-0.866025,5.000000e-01,0.433884,-0.900969,2016-01-07
978803,2,2016-01-21 00:41:54,1,1.41,-73.890945,40.746464,-73.887543,40.762894,8.3,1,21,3,0,US,0.000000,1.000000e+00,0.433884,-0.900969,2016-01-21
452829,2,2016-01-08 13:53:04,1,1.29,-73.867073,40.831944,-73.882149,40.844929,7.3,1,8,4,13,US,-0.258819,-9.659258e-01,-0.433884,-0.900969,2016-01-08
49731,2,2016-01-04 17:13:18,1,0.08,-73.935860,40.806469,-73.936714,40.805313,-4.3,1,4,0,17,US,-0.965926,-2.588190e-01,0.000000,1.000000,2016-01-04
231441,2,2016-01-29 18:06:02,1,1.79,-73.831833,40.715412,-73.816788,40.724331,9.8,1,29,4,18,US,-1.000000,-1.836970e-16,-0.433884,-0.900969,2016-01-29


### Enrich with Holiday Data

Now that the taxi data is downloaded and roughly prepared, add in holiday data as additional features. Holiday-specific features will assist model accuracy, as major holidays are times where taxi demand increases dramatically and supply becomes limited. The holiday dataset is relatively small, so fetch the full set by using the `PublicHolidays` class constructor with no parameters for filtering. Preview the data to check the format.

In [8]:
from azureml.opendatasets import PublicHolidays

# call default constructor to download full dataset
holidays_df = PublicHolidays().to_pandas_dataframe()
holidays_df.head(5)

[Info] read from C:\Users\FKHOSH~1\AppData\Local\Temp\tmppz6hd3q8\https%3A\%2Fazureopendatastorage.azurefd.net\holidaydatacontainer\Processed\part-00000-tid-8468414522853579044-35925ba8-a227-4b80-9c89-17065e7bf1db-649-c000.snappy.parquet


,countryOrRegion,holidayName,normalizeHolidayName,isPaidTimeOff,countryRegionCode,date
19375,Argentina,Año Nuevo [New Year's Day],Año Nuevo [New Year's Day],None,AR,2008-01-01
19376,Australia,New Year's Day,New Year's Day,None,AU,2008-01-01
19377,Austria,Neujahr,Neujahr,None,AT,2008-01-01
19378,Belarus,Новый год,Новый год,None,BY,2008-01-01
19379,Belgium,Nieuwjaarsdag,Nieuwjaarsdag,None,BE,2008-01-01


Rename the `countryRegionCode` and `date` columns to match the respective field names from the taxi data, and also normalize the time so it can be used as a key. Next, join the holiday data with the taxi data by performing a left-join using the Pandas `merge()` function. This will preserve all records from `green_taxi_df`, but add in holiday data where it exists for the corresponding `datetime` and `country_code`, which in this case is always `\"US\"`. Preview the data to verify that they were merged correctly.

In [9]:
holidays_df = holidays_df.rename(columns={"countryRegionCode": "country_code"})
holidays_df["datetime"] = holidays_df["date"].dt.normalize()

holidays_df.drop(["countryOrRegion", "holidayName", "date"], axis=1, inplace=True)

taxi_holidays_df = pd.merge(green_taxi_df, holidays_df, how="left", on=["datetime", "country_code"])
taxi_holidays_df[taxi_holidays_df["normalizeHolidayName"].notnull()]

,vendorID,lpepPickupDatetime,passengerCount,tripDistance,pickupLongitude,pickupLatitude,dropoffLongitude,dropoffLatitude,totalAmount,month_num,...,day_of_week,hour_of_day,country_code,hr_sin,hr_cos,dy_sin,dy_cos,datetime,normalizeHolidayName,isPaidTimeOff
31,2,2016-01-18 11:07:48,1,5.86,-73.966698,40.804382,-73.994469,40.744621,26.16,1,...,0,11,US,0.258819,-0.965926,0.000000,1.000000,2016-01-18,Martin Luther King Jr. Day,None
33,2,2016-01-01 23:27:20,1,3.38,-73.994156,40.681324,-73.938362,40.692184,18.59,1,...,4,23,US,-0.258819,0.965926,-0.433884,-0.900969,2016-01-01,New Year's Day,True
36,2,2016-01-18 00:24:15,1,1.39,-73.961540,40.714058,-73.954514,40.731960,8.30,1,...,0,0,US,0.000000,1.000000,0.000000,1.000000,2016-01-18,Martin Luther King Jr. Day,None
50,2,2016-01-18 23:47:50,1,1.10,-73.964813,40.682980,-73.961151,40.668526,7.30,1,...,0,23,US,-0.258819,0.965926,0.000000,1.000000,2016-01-18,Martin Luther King Jr. Day,None
62,2,2016-01-01 02:22:21,1,9.73,-73.991379,40.663376,-73.936852,40.751099,36.96,1,...,4,2,US,0.500000,0.866025,-0.433884,-0.900969,2016-01-01,New Year's Day,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23880,2,2016-12-25 19:36:27,1,5.20,NaN,NaN,NaN,NaN,23.30,12,...,6,19,US,-0.965926,0.258819,-0.781831,0.623490,2016-12-25,Christmas Day,True
23887,1,2016-12-25 00:57:40,1,1.30,NaN,NaN,NaN,NaN,9.35,12,...,6,0,US,0.000000,1.000000,-0.781831,0.623490,2016-12-25,Christmas Day,True
23928,1,2016-12-26 15:16:41,3,1.40,NaN,NaN,NaN,NaN,7.50,12,...,0,15,US,-0.707107,-0.707107,0.000000,1.000000,2016-12-26,Christmas Day,True
23979,2,2016-12-26 14:04:35,1,0.89,NaN,NaN,NaN,NaN,7.30,12,...,0,14,US,-0.500000,-0.866025,0.000000,1.000000,2016-12-26,Christmas Day,True


### Enrich with weather data

Now NOAA surface weather data can be appended to the taxi and holiday data. Use a similar approach to fetch the weather data by downloading one month at a time iteratively. Additionally, specify the `cols` parameter with an array of strings to filter the columns to download. This is a very large dataset containing weather surface data from all over the world, so before appending each month, filter the lat/long fields to near NYC using the `query()` function on the dataframe. This will ensure the `weather_df` doesn't get too large.

In [10]:
from azureml.opendatasets import NoaaIsdWeather
start = datetime.strptime("1/1/2016","%m/%d/%Y")
end = datetime.strptime("1/31/2016","%m/%d/%Y")

weather_df = pd.concat([NoaaIsdWeather(cols=["temperature", "precipTime", "precipDepth"], start_date=start + relativedelta(months=x), end_date=end + relativedelta(months=x))\
        .to_pandas_dataframe().query("latitude>=40.53 and latitude<=40.88 and longitude>=-74.09 and longitude<=-73.72 and temperature==temperature") for x in range(12)])

[Info] read from C:\Users\FKHOSH~1\AppData\Local\Temp\tmp5yuq8epd\https%3A\%2Fazureopendatastorage.azurefd.net\isdweatherdatacontainer\ISDWeather\year=2016\month=1\part-00000-tid-738723883827836859-85e0759b-51fa-4430-84ec-cb8ab6a57033-2623-1.c000.snappy.parquet
[Info] read from C:\Users\FKHOSH~1\AppData\Local\Temp\tmp5yuq8epd\https%3A\%2Fazureopendatastorage.azurefd.net\isdweatherdatacontainer\ISDWeather\year=2016\month=1\part-00001-tid-738723883827836859-85e0759b-51fa-4430-84ec-cb8ab6a57033-2628-1.c000.snappy.parquet
[Info] read from C:\Users\FKHOSH~1\AppData\Local\Temp\tmp5yuq8epd\https%3A\%2Fazureopendatastorage.azurefd.net\isdweatherdatacontainer\ISDWeather\year=2016\month=1\part-00002-tid-738723883827836859-85e0759b-51fa-4430-84ec-cb8ab6a57033-2622-1.c000.snappy.parquet
[Info] read from C:\Users\FKHOSH~1\AppData\Local\Temp\tmp5yuq8epd\https%3A\%2Fazureopendatastorage.azurefd.net\isdweatherdatacontainer\ISDWeather\year=2016\month=1\part-00003-tid-738723883827836859-85e0759b-51fa-44

In [11]:
weather_df

,precipTime,latitude,precipDepth,datetime,usaf,temperature,longitude,wban
204647,NaN,40.783,NaN,2016-01-02 03:00:00,725030,2.8,-73.867,14732
204670,1.0,40.779,0.0,2016-01-22 13:51:00,725030,-4.4,-73.880,14732
204694,1.0,40.779,0.0,2016-01-08 02:51:00,725030,5.0,-73.880,14732
204701,1.0,40.779,0.0,2016-01-04 15:51:00,725030,-1.1,-73.880,14732
204715,1.0,40.779,0.0,2016-01-01 21:51:00,725030,4.4,-73.880,14732
...,...,...,...,...,...,...,...,...
1248471,1.0,40.789,0.0,2016-12-23 13:51:00,725053,4.4,-73.967,94728
1248555,1.0,40.789,0.0,2016-12-12 13:51:00,725053,5.0,-73.967,94728
1248580,NaN,40.789,NaN,2016-12-18 07:01:00,725053,3.9,-73.967,94728
1248597,1.0,40.789,0.0,2016-12-25 00:51:00,725053,7.8,-73.967,94728


Again call `pandas.Series.dt.normalize` on the `datetime` field in the weather data so it matches the time key in `taxi_holidays_df`.


Next group the weather data to have daily aggregated weather values. Define a dict `aggregations` to define how to aggregate each field at a daily level. For`temperature` take the mean and for `precipTime` and `precipDepth` take the daily maximum. Use the `groupby()` function along with the aggregations to group the data. Preview the data to ensure there is one record per day.

In [12]:
weather_df["datetime"] = weather_df["datetime"].dt.normalize()

# group by datetime
aggregations = {"precipTime": "max", "temperature": "mean", "precipDepth": "max"}
weather_df_grouped = weather_df.groupby("datetime").agg(aggregations)
weather_df_grouped.head(10)

,precipTime,temperature,precipDepth
datetime,,,
2016-01-01,1.0,5.197345,0.0
2016-01-02,1.0,2.567857,0.0
2016-01-03,1.0,3.846429,0.0
2016-01-04,1.0,0.123894,0.0
2016-01-05,6.0,-7.206250,0.0
2016-01-06,6.0,-0.896396,0.0
2016-01-07,6.0,3.180645,0.0
2016-01-08,1.0,4.384091,0.0
2016-01-09,6.0,6.710274,3.0


Note: The examples in this tutorial merge data using Pandas functions and custom aggregations, but the Open Datasets SDK has classes designed to easily merge and enrich data sets. See the [notebook](https://github.com/Azure/OpenDatasetsNotebooks/blob/master/tutorials/data-join/04-nyc-taxi-join-weather-in-pandas.ipynb) for code examples of these design patterns.

### Cleanse data

Merge the existing taxi and holiday data with the new weather data. This time `datetime` is the only key, and again perform a left-join of the data. Run the `describe()` function on the new dataframe to see summary statistics for each field.

In [13]:
taxi_holidays_weather_df = pd.merge(taxi_holidays_df, weather_df_grouped, how="left", on=["datetime"])
taxi_holidays_weather_df.describe()

,vendorID,passengerCount,tripDistance,pickupLongitude,pickupLatitude,dropoffLongitude,dropoffLatitude,totalAmount,month_num,day_of_month,day_of_week,hour_of_day,hr_sin,hr_cos,dy_sin,dy_cos,precipTime,temperature,precipDepth
count,24000.000000,24000.000000,24000.000000,12000.000000,12000.000000,12000.000000,12000.000000,24000.000000,24000.000000,24000.000000,24000.000000,24000.000000,24000.000000,2.400000e+04,24000.000000,24000.000000,24000.000000,24000.000000,24000.000000
mean,1.784917,1.361583,2.800839,-73.819434,40.682874,-73.861534,40.704904,14.603318,6.500000,14.982083,3.249208,13.537500,-0.239262,-2.217893e-02,-0.083478,-0.054648,13.306292,13.944374,1039.118917
std,0.410889,1.028292,2.913202,2.940104,1.621160,2.337503,1.289197,11.815196,3.452124,8.442693,1.961699,6.689236,0.668796,7.035712e-01,0.710977,0.696131,10.316520,9.432455,2796.099709
min,1.000000,0.000000,0.000000,-74.098701,0.000000,-74.612396,0.000000,-450.000000,1.000000,1.000000,0.000000,0.000000,-1.000000,-1.000000e+00,-0.974928,-0.900969,1.000000,-13.379464,0.000000
25%,2.000000,1.000000,1.040000,-73.960459,40.694413,-73.967955,40.695667,7.800000,3.750000,8.000000,2.000000,9.000000,-0.866025,-7.071068e-01,-0.781831,-0.900969,6.000000,6.710274,0.000000
50%,2.000000,1.000000,1.810000,-73.945953,40.746107,-73.945446,40.746067,11.300000,6.500000,15.000000,3.000000,15.000000,-0.500000,-1.836970e-16,0.000000,-0.222521,6.000000,13.125893,10.000000
75%,2.000000,1.000000,3.500000,-73.918839,40.801971,-73.912586,40.791223,17.750000,9.250000,22.000000,5.000000,19.000000,0.258819,7.071068e-01,0.433884,0.623490,24.000000,22.892391,127.000000
max,2.000000,8.000000,69.780000,0.000000,40.919670,0.000000,40.967735,457.800000,12.000000,30.000000,6.000000,23.000000,1.000000,1.000000e+00,0.974928,1.000000,24.000000,31.303665,9999.000000


From the summary statistics, you see that there are several fields that have outliers or values that will reduce model accuracy. First filter the lat/long fields to be within the same bounds you used for filtering weather data. The `tripDistance` field has some bad data, because the minimum value is negative. The `passengerCount` field has bad data as well, with the max value being 210 passengers. Lastly, the `totalAmount` field has negative values, which don't make sense in the context of our model.

Filter out these anomolies using query functions, and then remove the last few columns unnecesary for training.

Note: since a random sample of 2000 was taken for each month of the taxi data, the statistics may vary each time this is ran.

In [14]:
final_df = taxi_holidays_weather_df.query("pickupLatitude>=40.53 and pickupLatitude<=40.88 and \
                                           pickupLongitude>=-74.09 and pickupLongitude<=-73.72 and \
                                           tripDistance>0 and tripDistance<75 and \
                                           passengerCount>0 and passengerCount<100 and \
                                           totalAmount>0")

Call `describe()` again on the data to ensure cleansing worked as expected. The final data is prepared and cleansed, consisting of taxi, holiday, and weather data, and is ready to use for machine learning model training.

In [15]:
final_df.describe()

,vendorID,passengerCount,tripDistance,pickupLongitude,pickupLatitude,dropoffLongitude,dropoffLatitude,totalAmount,month_num,day_of_month,day_of_week,hour_of_day,hr_sin,hr_cos,dy_sin,dy_cos,precipTime,temperature,precipDepth
count,11773.000000,11773.000000,11773.000000,11773.000000,11773.000000,11773.000000,11773.000000,11773.000000,11773.000000,11773.000000,11773.000000,11773.000000,11773.000000,1.177300e+04,11773.000000,11773.000000,11773.000000,11773.000000,11773.000000
mean,1.783063,1.359212,2.832306,-73.936961,40.746886,-73.910749,40.731292,14.561464,3.503440,14.993884,3.226960,13.561199,-0.237529,-1.768373e-02,-0.074460,-0.052581,11.780685,10.355071,200.826807
std,0.412177,1.021261,2.898582,0.041440,0.056204,1.363547,0.753125,10.508792,1.706435,8.415263,1.962474,6.713629,0.666515,7.064734e-01,0.711426,0.696885,10.081126,8.510100,1262.019432
min,1.000000,1.000000,0.010000,-74.040131,40.575253,-74.182785,0.000000,0.400000,1.000000,1.000000,0.000000,0.000000,-1.000000,-1.000000e+00,-0.974928,-0.900969,1.000000,-13.379464,0.000000
25%,2.000000,1.000000,1.080000,-73.960785,40.694481,-73.968376,40.695599,7.800000,2.000000,8.000000,2.000000,9.000000,-0.866025,-7.071068e-01,-0.781831,-0.900969,1.000000,3.504580,0.000000
50%,2.000000,1.000000,1.850000,-73.946632,40.745968,-73.945908,40.745827,11.300000,4.000000,15.000000,3.000000,15.000000,-0.500000,-1.836970e-16,0.000000,-0.222521,6.000000,10.468276,3.000000
75%,2.000000,1.000000,3.560000,-73.919167,40.801727,-73.913460,40.790363,17.760000,5.000000,22.000000,5.000000,19.000000,0.258819,7.071068e-01,0.781831,0.623490,24.000000,17.239744,41.000000
max,2.000000,6.000000,69.780000,-73.745674,40.879841,0.000000,40.967735,324.800000,6.000000,30.000000,6.000000,23.000000,1.000000,1.000000e+00,0.974928,1.000000,24.000000,26.524107,9999.000000


## Train a model

The data is ready to train a machine learning model.

In [20]:
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import RidgeCV
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.metrics import r2_score, mean_absolute_percentage_error, mean_squared_error

### Training Function

Define a function that can be used to create a model pipeline that can be trained and then used for scoring. This pipeline has 2 steps: preprocessing and model training.

<b>Preprocessing Stages:</b>
The preprocessing step of the pipeline also has 2 stages, one for numerical features and one for categorical features.
For the numerical features, let's fill in any blanks with 0's. While the training data may not have any nulls in the these fields, future data that is scored may and this step will take care of those for us. Optionally, a scaler transformation could be added in this step as well. Similarly for the categorical variables, let's have the null values filled with "MISSING". Additionally to the categorical variables, these will need to be one hot encoded, so we will include that step in our pipeline.

<b>Model Training Stage:</b>
An input parameter will determine which type of model of train. Let's test out a linear regression and random forest model to start. 

The two steps are put together into the pipeline which is what the function is returning.

In [27]:
def createClassModel(algo_name, catg, nums):
  numeric_transformer = Pipeline(steps=[('imputer', SimpleImputer(strategy='constant', fill_value=0))])
  
  categorical_transformer = Pipeline(steps=[('imputer', SimpleImputer(strategy='constant', fill_value="MISSING")), ('onehot', OneHotEncoder(handle_unknown='ignore'))])
  
  preprocesser = ColumnTransformer(transformers=[('num', numeric_transformer, nums), ('cat', categorical_transformer, catg)])
  
  if algo_name == 'linear_regression':
    #model = LinearRegression()
    model = RidgeCV(alphas=[1e-4, 1e-3, 1e-2, 1e-1, 1, 10, 100, 1000])
  elif algo_name == 'random_forest':
    model = RandomForestRegressor()
  else:
    pass
  ModelPipeline = Pipeline(steps=[('preprocessor', preprocesser), ("model", model)])
  return ModelPipeline

Let's define the arguments that will be passed to the function. `catg_cols` is a list of the categorical variables that will be transformed in our processing step. `num_cols` is a list of the numerical variables that will be transformed in our processing step. Let's define the target column as `label` so it can be used in future steps as well.

In [18]:
catg_cols = ["vendorID", "month_num", "day_of_month", "normalizeHolidayName", "isPaidTimeOff"]
num_cols = ["passengerCount", "tripDistance", "precipTime", "temperature", "precipDepth", "hr_sin", "hr_cos", "dy_sin", "dy_cos"]
label = ["totalAmount"]

The training is ready to begin, but first, let's make sure that the categorical variables are strings in our dataframe to ensure no errors in our pipeline. 

Next, the data is split into training and test sets by using the `train_test_split()` function in the `scikit-learn` library. The `test_size` parameter determines the percentage of data to allocate to testing. The `random_state` parameter sets a seed to the random number generator, so that your train-test splits are deterministic.

The training will happen in the for loop so that both algorithms can be tested. The createClassModel funtion is called to retreive the pipeline that can then be trained using the training dataset. 

Once trained, the test dataset is then ran through the model to test the model's performance. Using various functions from sklearn.metrics, the R2 score, MAPE, and RMSE can be used to measure model performance.

In [28]:
# make sure categorical columns are strings
final_df[catg_cols] = final_df[catg_cols].astype("str")

# split data
X_train, X_test, y_train, y_test = train_test_split(final_df.drop(label, axis=1), final_df[label], test_size=0.2, random_state=222)

# test 2 algorithms
for algorithmname in ["linear_regression"]:
    fitPipeline = createClassModel(algorithmname, catg_cols, num_cols) # get pipeline
    fitPipeline.fit(X_train, y_train.values.ravel())                   # fit pipeine

    y_pred = fitPipeline.predict(X_test)                               # score with fitted pipeline

    # Evaluate
    r2 = r2_score(y_test, y_pred)
    mape = mean_absolute_percentage_error(y_test, y_pred)
    rmse = np.sqrt(mean_squared_error(y_test, y_pred))

    print(algorithmname)
    print("R2:", r2)
    print("MAPE:", mape)
    print("RMSE:", rmse)
    print()

linear_regression
R2: 0.7921916148005586
MAPE: 0.1492697724713092
RMSE: 4.692283877328783

